# **Homework 2**

In [10]:
import gdown
import zipfile
import os

In [11]:
#Check if files exists in data folder
if os.path.exists('data/'):
    print('Files already downloaded.')

else:
    output_path = 'data.zip'
    file_id = '1KDN-rFCq9IDJ7_kNW5y5Co100KNpklz-'
    url = f'https://drive.google.com/uc?id={file_id}'
    # Download the zip file
    gdown.download(url, output_path, quiet=False)

    # Extract the contents of the zip file
    with zipfile.ZipFile(output_path, 'r') as zip_ref:
        zip_ref.extractall('data')

    # Remove the zip file
    os.remove(output_path)


Files already downloaded.


In [12]:
if(os.path.exists('test/') and os.path.exists('train')):
    print('Files already extracted')
else:
    print('Extracting the test.zip and train.zip files...')
    # Extract the test.zip file
    with zipfile.ZipFile('data/public/test.zip', 'r') as zip_ref:
        zip_ref.extractall()

    # Extract the train.zip file
    with zipfile.ZipFile('data/public/train.zip', 'r') as zip_ref:
        zip_ref.extractall()

    print('Done!')

Files already extracted


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# Step 1: Data Loading and Preprocessing
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(96),   # Randomly resize and crop the image
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust color
    transforms.RandomRotation(30),       # Randomly rotate the image
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Define data transformations for validation (you can modify these based on your needs)
val_transform = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load your dataset (adjust this based on your dataset structure)
train_dataset = ImageFolder(root='train', transform=train_transform)
val_dataset = ImageFolder(root='test', transform=val_transform)

num_workers = 2

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=num_workers)

# Print shapes of one batch of training and validation data
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break


Image batch dimensions: torch.Size([64, 3, 96, 96])
Image label dimensions: torch.Size([64])


In [42]:
from torchvision.models import resnet18

class CNNModel(nn.Module):
    def __init__(self, resnet_features):
        super(CNNModel, self).__init__()
        self.resnet_features = resnet_features
        self.fc1 = nn.Linear(512, 128)  # Assuming ResNet18 output size is 512
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(128, 96)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.4)
        self.output = nn.Linear(96, 5)  # Assuming 5 classes
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet_features(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.output(x)
        x = self.softmax(x)
        return x

# Load pre-trained ResNet18 model and extract features
resnet_model = resnet18(pretrained=True)
resnet_features = nn.Sequential(*list(resnet_model.children())[:-1])

# Create your model instance
model = CNNModel(resnet_features)

# Example forward pass
image_batch = torch.randn(64, 3, 96, 96)  # Example image batch
label_batch = torch.randint(5, (64,))  # Example label batch
output = model(image_batch)

In [43]:
criterion = nn.CrossEntropyLoss()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion.to(device)

# Optimizer and Scheduler
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Optional learning rate scheduler


In [45]:
# Step 3: Training loop
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:  # Assuming you have a DataLoader for your training set
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()

    # Print the average loss for the epoch
    average_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')


Epoch 1/50, Loss: 1.4443379402160645
Epoch 2/50, Loss: 1.4410130310058593
Epoch 3/50, Loss: 1.4449812841415406
Epoch 4/50, Loss: 1.4332814645767211
Epoch 5/50, Loss: 1.4332589673995972
Epoch 6/50, Loss: 1.4305340230464936
Epoch 7/50, Loss: 1.4284508514404297
Epoch 8/50, Loss: 1.4241294944286347
Epoch 9/50, Loss: 1.431456551551819
Epoch 10/50, Loss: 1.431884664297104
Epoch 11/50, Loss: 1.4269499504566192
Epoch 12/50, Loss: 1.4266185009479522
Epoch 13/50, Loss: 1.4224174201488495
Epoch 14/50, Loss: 1.419676307439804
Epoch 15/50, Loss: 1.4240376591682433
Epoch 16/50, Loss: 1.4177034974098206
Epoch 17/50, Loss: 1.418025212287903
Epoch 18/50, Loss: 1.423771276473999
Epoch 19/50, Loss: 1.4171944236755372
Epoch 20/50, Loss: 1.4189593636989593
Epoch 21/50, Loss: 1.4239470970630645
Epoch 22/50, Loss: 1.4191369318962097
Epoch 23/50, Loss: 1.4219366466999055
Epoch 24/50, Loss: 1.4197205317020416
Epoch 25/50, Loss: 1.4165246164798737
Epoch 26/50, Loss: 1.4243426752090453
Epoch 27/50, Loss: 1.42209

In [46]:

# After training, you can evaluate the model on your validation set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:  # Assuming you have a DataLoader for your validation set
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy}')


Validation Accuracy: 0.6569661695161877


In [8]:
torch.save(model.state_dict(), 'MyCNN.pth')

In [9]:

# Step 5: Model Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy}')


# Repeat steps 2-5 with a different architecture for Approach B
# ...

# Provide analysis and comments
# ...

Validation Accuracy: 0.6711531465987632
